In [240]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor

In [241]:
dataset = pd.read_excel('../data/boxScores.xlsx')


In [242]:
# Filter out rows where the 'MIN' column is zero
dataset = dataset[dataset['MIN'] != 0]
dataset = dataset.dropna(subset=['W/L', 'Last3_FP_Avg'])
dataset.describe()

,GAME DATE,MIN,PTS,FGM,FGA,3PM,3PA,FTM,FTA,OREB,...,BLK,TOV,PF,+/-,FP,CLUSTER,Last3_FP_Avg,Last5_FP_Avg,Last7_FP_Avg,Season_FP_Avg
count,9892,9892.000000,9892.000000,9892.000000,9892.000000,9892.000000,9892.000000,9892.000000,9892.000000,9892.000000,...,9892.000000,9892.000000,9892.000000,9892.000000,9892.000000,7562.000000,9892.000000,9892.000000,9892.000000,9892.000000
mean,2024-11-25 11:29:17.056206848,22.739790,10.652042,3.884958,8.366154,1.276890,3.550344,1.605237,2.057016,1.022038,...,0.481702,1.331076,1.797614,0.012535,21.852911,7.623777,21.646553,21.545376,21.469300,21.164617
min,2024-10-24 00:00:00,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,-43.000000,-3.000000,0.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,2024-11-08 00:00:00,15.000000,4.000000,1.000000,4.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,-7.000000,9.900000,4.000000,11.645833,11.900000,11.885714,11.674432
50%,2024-11-24 00:00:00,24.000000,9.000000,3.000000,7.000000,1.000000,3.000000,1.000000,1.000000,1.000000,...,0.000000,1.000000,2.000000,0.000000,20.000000,8.000000,20.200000,20.160000,20.046429,19.830331
75%,2024-12-13 00:00:00,32.000000,16.000000,6.000000,12.000000,2.000000,5.000000,2.000000,3.000000,2.000000,...,1.000000,2.000000,3.000000,7.000000,31.400000,11.000000,30.066667,29.840000,29.614286,29.109821
max,2024-12-31 00:00:00,48.000000,60.000000,22.000000,39.000000,10.000000,20.000000,17.000000,26.000000,12.000000,...,10.000000,10.000000,6.000000,43.000000,98.300000,14.000000,80.500000,75.360000,72.200000,72.200000
std,NaN,10.740598,8.828397,3.241424,6.027972,1.539577,3.177172,2.249332,2.716658,1.352351,...,0.846630,1.457750,1.446824,11.398963,14.976485,4.277608,13.112352,12.707586,12.541914,12.269812


In [243]:
# Filter out players who aren't in clusters for accuracy
clusterdf = pd.read_excel('../data/clusteredPlayers.xlsx')
clusterdf = clusterdf.drop('Unnamed: 0', axis=1)
clusterDict = {}
clusterDict.update(pd.Series(clusterdf['Cluster'].values, index=clusterdf['Player']).to_dict())
dataset['CLUSTER'] = dataset['PLAYER'].map(clusterDict).astype('category')
dataset = dataset.dropna(subset=['CLUSTER'])
dataset.sort_values(by=['GAME DATE'], ascending = [False], inplace=True)
len(dataset)

7562

In [244]:
dataset.describe()

,GAME DATE,MIN,PTS,FGM,FGA,3PM,3PA,FTM,FTA,OREB,...,STL,BLK,TOV,PF,+/-,FP,Last3_FP_Avg,Last5_FP_Avg,Last7_FP_Avg,Season_FP_Avg
count,7562,7562.000000,7562.000000,7562.000000,7562.000000,7562.000000,7562.000000,7562.000000,7562.000000,7562.000000,...,7562.000000,7562.000000,7562.000000,7562.000000,7562.000000,7562.000000,7562.000000,7562.000000,7562.000000,7562.000000
mean,2024-11-25 06:30:10.949484288,26.564136,12.785110,4.653663,9.979635,1.536498,4.234065,1.941285,2.466014,1.156440,...,0.932293,0.551045,1.568633,2.042978,0.229569,25.888151,25.686137,25.590725,25.523841,25.202230
min,2024-10-24 00:00:00,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,-43.000000,-2.800000,-1.000000,-1.000000,-1.000000,-1.000000
25%,2024-11-08 00:00:00,20.000000,6.000000,2.000000,6.000000,0.000000,2.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,-8.000000,15.300000,16.966667,17.125000,17.242857,17.029762
50%,2024-11-24 00:00:00,27.000000,11.000000,4.000000,9.000000,1.000000,4.000000,1.000000,2.000000,1.000000,...,1.000000,0.000000,1.000000,2.000000,0.000000,24.500000,24.200000,24.160000,24.100000,23.706071
75%,2024-12-13 00:00:00,33.000000,18.000000,7.000000,14.000000,2.000000,6.000000,3.000000,4.000000,2.000000,...,1.000000,1.000000,2.000000,3.000000,8.000000,34.700000,33.033333,32.615000,32.471429,31.991365
max,2024-12-31 00:00:00,48.000000,60.000000,22.000000,39.000000,10.000000,20.000000,17.000000,26.000000,12.000000,...,8.000000,10.000000,10.000000,6.000000,43.000000,98.300000,80.500000,75.360000,72.200000,72.200000
std,NaN,8.497846,8.740886,3.214172,5.787386,1.613264,3.215237,2.395714,2.876227,1.414584,...,1.059987,0.899440,1.520781,1.427943,12.356974,14.184908,11.924989,11.418716,11.201298,10.879706


For reference, features are the raw input data provided to the model to generate predictions, while the label is the target outcome that the model aims to predict. These above features can and probably should be tweaked and adjusted to improve the model's performance.

In [245]:
featureNames = ['Last3_FP_Avg', 'Last5_FP_Avg', 'Last7_FP_Avg', 'Season_FP_Avg']
labelName = ['FP']
dfFeatures = dataset[featureNames]
dfLabels = dataset[labelName]

In [246]:
labels = np.array(dfLabels)
features = np.array(dfFeatures)

** Note numpy array is only numbers while pandas dataframe can be a mix of numbers and strings.

In [247]:
# Scaling the data
x = features
x = StandardScaler().fit_transform(x)

In [248]:
dataset = dataset[['PLAYER', 'GAME DATE', 'FP', 'Last3_FP_Avg', 'Last5_FP_Avg', 'Last7_FP_Avg', 'Season_FP_Avg']]
players = dataset['PLAYER']
gameDates = dataset['GAME DATE']

In [249]:
train, test, trainLabels, testLabels, trainPlayers, testPlayers, trainDates, testDates= train_test_split(features, labels, players, gameDates, test_size=0.25, random_state=4)
testPlayers = testPlayers.reset_index(drop=True)
testDates = testDates.reset_index(drop=True)


In [250]:
rf = RandomForestRegressor(random_state=4)

In [251]:
import pickle
rf.fit(train, trainLabels)
with open("../models/RFFullNBA.sav", 'wb') as f:
    pickle.dump(rf, f)

c:\Users\Ian Bracken\Desktop\WebDev\nba-prediction-model\venv\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [252]:
train_predictions = rf.predict(train)
predictions = rf.predict(test)

In [253]:
df = pd.DataFrame(test, columns = featureNames)
df['PLAYER'] = testPlayers
df['GAME DATE'] = testDates
df['actual'] = testLabels
df['predicted'] = predictions
df['error'] = abs(df['actual'] - df['predicted'])
df = df[['PLAYER'] + [col for col in df.columns if col != 'PLAYER']]

In [254]:
df.describe()

,Last3_FP_Avg,Last5_FP_Avg,Last7_FP_Avg,Season_FP_Avg,GAME DATE,actual,predicted,error
count,1891.000000,1891.000000,1891.000000,1891.000000,1891,1891.000000,1891.000000,1891.000000
mean,25.591653,25.685279,25.666322,25.286577,2024-11-25 01:50:25.066102528,26.233474,25.945908,8.824187
min,0.000000,0.000000,0.000000,0.000000,2024-10-24 00:00:00,-2.800000,5.831500,0.002833
25%,16.866667,17.350000,17.561905,17.292308,2024-11-08 00:00:00,15.800000,18.164250,3.512958
50%,23.700000,23.960000,23.900000,23.566667,2024-11-24 00:00:00,25.000000,24.300500,7.423000
75%,32.733333,32.500000,32.714286,31.854048,2024-12-13 00:00:00,35.000000,32.012033,12.417500
max,80.500000,75.360000,68.314286,67.636842,2024-12-31 00:00:00,88.900000,67.171000,44.452000
std,11.836829,11.352353,11.124797,10.777226,NaN,14.271164,10.261654,6.853645


In [255]:
train, test, trainLabels, testLabels = train_test_split(x, labels, test_size=0.25, random_state=4)
rf = RandomForestRegressor(random_state=4)
rf.fit(train, trainLabels)
rfPredNoStandard = rf.predict(test)

c:\Users\Ian Bracken\Desktop\WebDev\nba-prediction-model\venv\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [256]:
df4 = pd.DataFrame(test, columns = featureNames)
df4['actual'] = testLabels
df4['predicted'] = rfPredNoStandard
df4['error'] = abs(df4['actual'] - df4['predicted'])

In [257]:
df4.describe()

,Last3_FP_Avg,Last5_FP_Avg,Last7_FP_Avg,Season_FP_Avg,actual,predicted,error
count,1891.000000,1891.000000,1891.000000,1891.000000,1891.000000,1891.000000,1891.000000
mean,-0.007924,0.008281,0.012721,0.007753,26.233474,25.947728,8.829426
std,0.992673,0.994254,0.993236,0.990646,14.271164,10.262658,6.849280
min,-2.154118,-2.241269,-2.278801,-2.316597,-2.800000,5.463000,0.002833
25%,-0.739628,-0.721734,-0.710852,-0.727083,15.800000,18.145000,3.524250
50%,-0.166564,-0.142821,-0.144979,-0.150342,25.000000,24.307833,7.445000
75%,0.590999,0.605123,0.641972,0.611437,35.000000,32.032167,12.448875
max,4.596858,4.358858,3.820385,3.900603,88.900000,67.171000,44.452000


In [258]:
# pd.DataFrame.to_excel(df, '../data/playerPredictions.xlsx')

In [259]:
dfCheck = pd.DataFrame()
# Difference in accuracy between df3 and df4 is negligible so I am sticking with 3 for better readability
dfCheck['df3'] = df['error']
dfCheck['df4'] = df4['error']
dfCheck.describe()

,df3,df4
count,1891.000000,1891.000000
mean,8.824187,8.829426
std,6.853645,6.849280
min,0.002833,0.002833
25%,3.512958,3.524250
50%,7.423000,7.445000
75%,12.417500,12.448875
max,44.452000,44.452000


Note: Scaling and preprocessing data matters more on larger datasets